In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
import jax
from jax import numpy as jnp
from jax.experimental import host_callback

from tqdm.auto import tqdm
from functools import partial
import optax
import matplotlib.pyplot as plt
from pprint import pprint

from flax import linen as nn
from typing import Sequence

from functools import partial
from typing import Callable, Sequence
from flax import struct
from flax import linen as nn
import jax
import jax.numpy as jnp
import optax
from functools import partial
import numpy as np

import actions
import qlearning
import replay
import plotting
from utils import ConvNet, Transition
from frozen_lake import EnvState, FrozenLake, ObsType, ActType, RNGKey


In [3]:
jax.devices()

[cuda(id=0)]

In [4]:
import frozen_lake, utils, replay, qlearning
rng_key = jax.random.PRNGKey(0)
map_size = 8
rng_env_init, rng_env_reset, rng_dql = jax.random.split(rng_key, 3)
env = frozen_lake.FrozenLake.make(rng_env_init, (map_size, map_size))
env_state, env_obs = env.reset(rng_env_reset)

qnet = utils.ConvNet(hidden=[2 * map_size] * int(jnp.log2(map_size)), out=4)
dql_state = qlearning.DQLTrainState.create(rng_dql, qnet, env_obs)

init_transitions = qlearning.eps_greedy_rollout(env, dql_state, rng_key, epsilon=0.0, steps=2)
replay_memory = replay.CircularBuffer.create(init_transitions)

2024-02-01 08:58:32.137458: W external/xla/xla/service/gpu/conv_algorithm_picker.cc:810] None of the algorithms provided by cuDNN heuristics worked; trying fallback algorithms.
2024-02-01 08:58:32.137493: W external/xla/xla/service/gpu/conv_algorithm_picker.cc:813] Conv: (f32[1,16,8,8]{3,2,1,0}, u8[0]{0}) custom-call(f32[1,3,8,8]{3,2,1,0}, f32[16,3,3,3]{3,2,1,0}), window={size=3x3 pad=1_1x1_1}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convForward", backend_config={"conv_result_scale":1,"activation_mode":"kNone","side_input_scale":0,"leakyrelu_alpha":0}


XlaRuntimeError: UNKNOWN: Failed to determine best cudnn convolution algorithm for:
%cudnn-conv.1 = (f32[1,16,8,8]{3,2,1,0}, u8[0]{0}) custom-call(f32[1,3,8,8]{3,2,1,0} %transpose, f32[16,3,3,3]{3,2,1,0} %transpose.1), window={size=3x3 pad=1_1x1_1}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convForward", metadata={op_name="jit(conv_general_dilated)/jit(main)/conv_general_dilated[window_strides=(1, 1) padding=((1, 1), (1, 1)) lhs_dilation=(1, 1) rhs_dilation=(1, 1) dimension_numbers=ConvDimensionNumbers(lhs_spec=(0, 3, 1, 2), rhs_spec=(3, 2, 0, 1), out_spec=(0, 3, 1, 2)) feature_group_count=1 batch_group_count=1 precision=None preferred_element_type=None]" source_file="/home/davide_sartor/MANGO-lite/jax/utils.py" source_line=33}, backend_config={"conv_result_scale":1,"activation_mode":"kNone","side_input_scale":0,"leakyrelu_alpha":0}

Original error: INTERNAL: All algorithms tried for (f32[1,16,8,8]{3,2,1,0}, u8[0]{0}) custom-call(f32[1,3,8,8]{3,2,1,0}, f32[16,3,3,3]{3,2,1

In [ ]:
%timeit qlearning.eps_greedy_rollout(env, dql_state, rng_key, 0.5, 256)